In [56]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("restaurant-review-average")\
        .master('local[*]').getOrCreate()

In [57]:
sc = spark.sparkContext # 필요 시 RDD API를 위해 사용

In [6]:
sc.defaultParallelism

2

In [8]:
sc.setLogLevel('INFO')

In [10]:
sc.getConf().getAll()

[('spark.driver.host', '7778534245f4'),
 ('spark.driver.port', '43305'),
 ('spark.app.name', 'restaurant-review-average'),
 ('spark.app.startTime', '1754273604430'),
 ('spark.executor.id', 'driver'),
 ('spark.app.id', 'local-1754273604894'),
 ('spark.driver.extraJavaOptions',
  '-Djava.net.preferIPv6Addresses=false -XX:+IgnoreUnrecognizedVMOptions --add-opens=java.base/java.lang=ALL-UNNAMED --add-opens=java.base/java.lang.invoke=ALL-UNNAMED --add-opens=java.base/java.lang.reflect=ALL-UNNAMED --add-opens=java.base/java.io=ALL-UNNAMED --add-opens=java.base/java.net=ALL-UNNAMED --add-opens=java.base/java.nio=ALL-UNNAMED --add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.util.concurrent=ALL-UNNAMED --add-opens=java.base/java.util.concurrent.atomic=ALL-UNNAMED --add-opens=java.base/sun.nio.ch=ALL-UNNAMED --add-opens=java.base/sun.nio.cs=ALL-UNNAMED --add-opens=java.base/sun.security.action=ALL-UNNAMED --add-opens=java.base/sun.util.calendar=ALL-UNNAMED --add-opens=java.s

In [5]:
data = [
    (0, "짜장면", "중식", 125),
    (1, "짬뽕", "중식", 235),
    (2, "김밥", "분식", 32),
    (3, "떡볶이", "분식", 534),
    (4, "라멘", "일식", 223),
    (5, "돈가스", "일식", 52),
    (6, "우동", "일식", 12),
    (7, "쌀국수", "아시안", 312),
    (8, "햄버거", "패스트푸드", 12),
    (9, "치킨", "패스트푸드", 23),
]

In [11]:
rdd1 = sc.parallelize(data)

In [12]:
rdd1.take(5)

[(0, '짜장면', '중식', 125),
 (1, '짬뽕', '중식', 235),
 (2, '김밥', '분식', 32),
 (3, '떡볶이', '분식', 534),
 (4, '라멘', '일식', 223)]

In [58]:
lines = sc.textFile("file:///home/jovyan/work/start_spark/learning_spark_data/restaurant_reviews.csv")

lines.take(5)

['id,item,cateogry,reviews,',
 '0,짜장면,중식,125,',
 '1,짬뽕,중식,235,',
 '2,김밥,분식,32,',
 '3,떡볶이,분식,534,']

In [19]:
# 총 건수 확인
lines.count() - 1

10

In [59]:
def parse(row):
    fields = row.split(",")
    
    category = fields[2] #카테고리
    
    # reviews는 정수로 parse
    reviews = fields[3] # 리뷰수
    reviews = int(reviews)
    
    return category, reviews

In [60]:
# 첫줄(컬럼)과 데이터 분리
header = lines.first()
filtered_lines = lines.filter(lambda row:row != header)

In [63]:
category_reviews = filtered_lines.map(parse)
category_reviews

PythonRDD[5] at RDD at PythonRDD.scala:53

In [64]:
category_reviews.collect()

[('중식', 125),
 ('중식', 235),
 ('분식', 32),
 ('분식', 534),
 ('일식', 223),
 ('일식', 52),
 ('일식', 12),
 ('아시안', 312),
 ('패스트푸드', 12),
 ('패스트푸드', 23)]

map(key, value) -> key, value 전체에 적용<br>
mapValues(key, value) -> value에만 적용

In [41]:
category_review_count = category_reviews.mapValues(lambda x : (x, 1)) # x는 review 개수
category_review_count.collect()

[('중식', (125, 1)),
 ('중식', (235, 1)),
 ('분식', (32, 1)),
 ('분식', (534, 1)),
 ('일식', (223, 1)),
 ('일식', (52, 1)),
 ('일식', (12, 1)),
 ('아시안', (312, 1)),
 ('패스트푸드', (12, 1)),
 ('패스트푸드', (23, 1))]

In [43]:
#같은 key(카테고리)끼리 x와 y의 (리뷰 수 합계, 개수 합계)를 누적
#첫번째 중식 x, 두번째 중식 y , [0] 은 리뷰수, [1] 은 개수 -> 같은 키끼리 행을 바꿔가며 계속 누적해 나간다.
reduced = category_review_count.reduceByKey(lambda x, y : (x[0] + y[0], x[1] + y[1]))
reduced.collect()

[('중식', (360, 2)),
 ('분식', (566, 2)),
 ('일식', (287, 3)),
 ('아시안', (312, 1)),
 ('패스트푸드', (35, 2))]

In [45]:
average = reduced.mapValues(lambda x : round(x[0]/ x[1], 2))
average.collect()

[('중식', 180.0), ('분식', 283.0), ('일식', 95.67), ('아시안', 312.0), ('패스트푸드', 17.5)]

In [47]:
# Persist 를 사용하는 경우

In [65]:
categoryReviews = filtered_lines.map(parse).persist() # categoryReviews RDD는 하나만 존재하는 RDD
categoryReviews

PythonRDD[6] at RDD at PythonRDD.scala:53

In [67]:
categoryReviews.take(10)

[('중식', 125),
 ('중식', 235),
 ('분식', 32),
 ('분식', 534),
 ('일식', 223),
 ('일식', 52),
 ('일식', 12),
 ('아시안', 312),
 ('패스트푸드', 12),
 ('패스트푸드', 23)]

In [50]:
categoryReviews.mapValues(lambda x : (x, 1)).collect()

[('중식', (125, 1)),
 ('중식', (235, 1)),
 ('분식', (32, 1)),
 ('분식', (534, 1)),
 ('일식', (223, 1)),
 ('일식', (52, 1)),
 ('일식', (12, 1)),
 ('아시안', (312, 1)),
 ('패스트푸드', (12, 1)),
 ('패스트푸드', (23, 1))]

In [51]:
categoryReviews \
    .reduceByKey(lambda a, b: a + b) \
    .collect()

[('중식', 360), ('분식', 566), ('일식', 287), ('아시안', 312), ('패스트푸드', 35)]

In [68]:
categoryReviews.unpersist()

PythonRDD[6] at RDD at PythonRDD.scala:53

In [55]:
spark.stop()
sc.stop()

In [69]:
from pyspark import StorageLevel
categoryReviews.persist(StorageLevel.MEMORY_AND_DISK)

PythonRDD[6] at RDD at PythonRDD.scala:53

In [70]:
categoryReviews.take(3)

[('중식', 125), ('중식', 235), ('분식', 32)]

In [71]:
categoryReviews.unpersist()

PythonRDD[6] at RDD at PythonRDD.scala:53

In [ ]:
# Narrow Transformations
1:1 변환 -> 하나의 열을 다룰 때 다른 데이터가 필요 없는 경우
filter(), map(), flatMap(), sample(), union()

# wide transformations

# flatMap()

In [72]:
rdd = sc.parallelize([1, 2, 3])
rdd_map = rdd.map(lambda x: [x, x + 1])  # => [[1, 2], [2, 3], [3, 4]]
rdd_map.collect()

[[1, 2], [2, 3], [3, 4]]

In [73]:
rdd_flatmap = rdd.flatMap(lambda x: [x, x + 1])  # => [1, 2, 2, 3, 3, 4]
rdd_flatmap.collect()

[1, 2, 2, 3, 3, 4]

In [74]:
movies = [
    "그린 북",
    "매트릭스",
    "토이 스토리",
    "캐스트 어웨이",
    "포드 V 페라리",
    "보헤미안 랩소디",
    "빽 투 더 퓨처",
    "반지의 제왕",
    "죽은 시인의 사회"
]

In [75]:
moviesRDD = sc.parallelize(movies)
moviesRDD.take(5)

['그린 북', '매트릭스', '토이 스토리', '캐스트 어웨이', '포드 V 페라리']

In [76]:
movies_map = moviesRDD.map(lambda x : x.split(' '))
movies_map.take(5)

[['그린', '북'], ['매트릭스'], ['토이', '스토리'], ['캐스트', '어웨이'], ['포드', 'V', '페라리']]

In [77]:
movies_flatmap = moviesRDD.flatMap(lambda x : x.split(' '))
movies_flatmap.take(5)

['그린', '북', '매트릭스', '토이', '스토리']

In [ ]:
# 집합 Transformation

In [78]:
num1 = sc.parallelize([1,2,3,4,5])
num2 = sc.parallelize([4,5,6,7,8,9,10])

In [79]:
num1.intersection(num2).collect()

[4, 5]

In [ ]:
# 합집합 구하기 - union

In [84]:
num1.union(num2).collect()

[1, 2, 3, 4, 5, 4, 5, 6, 7, 8, 9, 10]

In [ ]:
# 차집합 구하기 - subtract

In [81]:
num1.subtract(num2).collect()

[1, 2, 3]

In [87]:
num_union = num1.union(num2)

In [103]:
# 데이터 랜덤 추출 - sample(withReplacement, fraction, seed=None)
# withReplacement : True -> 중복 추출
num_union.sample(True, 0.7).collect()

[1, 1, 3, 3, 5, 7, 8, 9, 10]

In [101]:
# withReplacement : False -> 중복 X
num_union.sample(False, 0.7).collect()

[1, 5, 4, 7, 8, 9]

In [106]:
# 랜덤을 고정해서 항상 같은 결과가 나올 수 있도록
num_union.sample(True, 0.5, seed=42).collect()

[4, 5, 5, 5, 7]

In [85]:
foods = sc.parallelize([
    "짜장면", "마라탕", "짬뽕", "떡볶이", "쌀국수", "짬뽕", "짜장면", "짜장면", "짜장면", "라면", "우동", "라면"
])
foods

ParallelCollectionRDD[61] at readRDDFromFile at PythonRDD.scala:289

In [86]:
foodsGroup = foods.groupBy(lambda x : x[0])
foodsGroup.collect()

[('짜', <pyspark.resultiterable.ResultIterable at 0x7ff5daa83710>),
 ('짬', <pyspark.resultiterable.ResultIterable at 0x7ff5d9ab0850>),
 ('쌀', <pyspark.resultiterable.ResultIterable at 0x7ff5d9703890>),
 ('라', <pyspark.resultiterable.ResultIterable at 0x7ff5d9943590>),
 ('우', <pyspark.resultiterable.ResultIterable at 0x7ff5d97035d0>),
 ('마', <pyspark.resultiterable.ResultIterable at 0x7ff5d97034d0>),
 ('떡', <pyspark.resultiterable.ResultIterable at 0x7ff5d9703550>)]

In [107]:
res = foodsGroup.collect()

for (k, v) in res:
    print(k, list(v))

짜 ['짜장면', '짜장면', '짜장면', '짜장면']
짬 ['짬뽕', '짬뽕']
쌀 ['쌀국수']
라 ['라면', '라면']
우 ['우동']
마 ['마라탕']
떡 ['떡볶이']


In [122]:
kv_rdd = sc.parallelize([("apple", 1), ("banana", 2), ("cherry", 3), ("banana",4)])

In [127]:
grouped_rdd = kv_rdd.groupByKey()
res = kvGroup.collect()

for (k, v) in res:
    print(k, list(v))

apple [('apple', 1)]
banana [('banana', 2), ('banana', 4)]
cherry [('cherry', 3)]


In [124]:
reduced_rdd = kv_rdd.reduceByKey(lambda x,y : x+y)
reduced_rdd.collect()

[('apple', 1), ('banana', 6), ('cherry', 3)]

In [132]:
rdd1 = sc.parallelize([("apple", 2), ("banana", 1), ("cherry", 3)])
rdd2 = sc.parallelize([("apple", "fruit"), ("banana", "fruit"), ("carrot", "vegetable")])

In [133]:
joined_rdd = rdd1.join(rdd2)
print(joined_rdd.collect())

[('apple', (2, 'fruit')), ('banana', (1, 'fruit'))]


In [134]:
left_joined = rdd1.leftOuterJoin(rdd2)
print(left_joined.collect())

[('apple', (2, 'fruit')), ('banana', (1, 'fruit')), ('cherry', (3, None))]


In [135]:
right_joined = rdd1.rightOuterJoin(rdd2)
print(right_joined.collect())

[('apple', (2, 'fruit')), ('banana', (1, 'fruit')), ('carrot', (None, 'vegetable'))]


In [136]:
subtract_result = rdd1.subtractByKey(rdd2)
print(subtract_result.collect())

[('cherry', 3)]


In [140]:
spark.stop()
sc.stop()